# Wstępna analiza danych o retinopatii cukrzycowej

Data Set został pobrany z repozytorium UCI (http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).
Dane umieszczone tam są w formacie arff. W celu obsługi formatu należy użyć funkcję z pakietu SciPy oraz pakietu Pandas dla większego komfortu przeglądania informacji.

In [44]:
from scipy.io import arff
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [45]:
data = arff.loadarff('rethinpathy.arff')
df = pd.DataFrame(data[0])

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,Class
0,1.0,1.0,22.0,22.0,22.0,19.0,18.0,14.0,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1.0,b'0'
1,1.0,1.0,24.0,24.0,22.0,18.0,16.0,13.0,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0.0,b'0'
2,1.0,1.0,62.0,60.0,59.0,54.0,47.0,33.0,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0.0,b'1'
3,1.0,1.0,55.0,53.0,53.0,50.0,43.0,31.0,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0.0,b'0'
4,1.0,1.0,44.0,44.0,44.0,41.0,39.0,27.0,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0.0,b'1'


## Oznaczenie kolumn
0    - Oznaczenie jakości. 0 = zła jakość, 1 = dobra jakość<br>
1    - Wynik poprzedniego badania tj. czy było robione coś wczesniej, z czym pacjent przychodzi do okulisty. 1 oznacza nieprawidłowość siatkówki<br>
2-7  - Wynik wykrywania mikrotętniaków, wyrażone przez ilość takich poszerzeń naczynek<br>
8-15 - Znormalizowany wynik badania ilości wysięków. Normalizacja następuje przez iloczyn średnicy obszaru przez obraz który się bada<br>
16   - Znormalizowana odległość od środka plamki żółtej do środka tarczy nerwu wzrokowego <br>
17   - Średnica tarczy nerwu wzrokowego<br>
18   - Wyniki klasyfikacji patologicznych obrazów siatkówki za pomocą metody wykorzystującej modulacje amplitudy i częstotliwości<br>
19   - 1 = występują oznaki retinopatii, 0 = nie występują

Należy sprawdzić w jakim typie są zapisane dane.

In [46]:
df.dtypes

0        float64
1        float64
2        float64
3        float64
4        float64
5        float64
6        float64
7        float64
8        float64
9        float64
10       float64
11       float64
12       float64
13       float64
14       float64
15       float64
16       float64
17       float64
18       float64
Class     object
dtype: object

Ponieważ część danych jest binarnych lub wyrażonych tylko liczbami naturalnymi, lepiej będzie gdy zostaną przedstawione tylko w postaci całkowitej

In [47]:
df[df.columns[0:8]] = df[df.columns[0:8]].astype(int)
df[df.columns[18:20]] = df[df.columns[18:20]].astype(int)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,Class
0,1,1,22,22,22,19,18,14,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1,0
1,1,1,24,24,22,18,16,13,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0,0
2,1,1,62,60,59,54,47,33,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0,1
3,1,1,55,53,53,50,43,31,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0,0
4,1,1,44,44,44,41,39,27,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0,1


Należy teraz nazwać kolumny by nadać sens zaimportowanemu zestawowi liczb.

In [48]:
col_names = ['Quality', 'Pre-screening', 'MA result1', 'MA result2', 'MA result3', 'MA result4', 'MA result5', 'MA result6', 'Exudates result1', 'Exudates result2', 'Exudates result3', 'Exudates result4', 'Exudates result5', 'Exudates result6', 'Exudates result7',
            'Exudates result', 'Macula - Optic disc dist', 'Optic disc diameter', 'AM/FM classification', 'Class']
df.columns = col_names
df.head()

,Quality,Pre-screening,MA result1,MA result2,MA result3,MA result4,MA result5,MA result6,Exudates result1,Exudates result2,Exudates result3,Exudates result4,Exudates result5,Exudates result6,Exudates result7,Exudates result,Macula - Optic disc dist,Optic disc diameter,AM/FM classification,Class
0,1,1,22,22,22,19,18,14,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1,0
1,1,1,24,24,22,18,16,13,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0,0
2,1,1,62,60,59,54,47,33,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0,1
3,1,1,55,53,53,50,43,31,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0,0
4,1,1,44,44,44,41,39,27,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0,1


In [49]:
df.isnull().values.any()

False

# 2 zadanie

By rozwiązać problem klasyfikacji danych należy odpowiednio przygotować dataset. Najpierw przypisuje do nowego dataframe'u wszystkie kolumny prócz ostatniej żeby oddzielić dane od wyniku.
Ostatnia kolumna jest zapisywana do oddzielnej zmiennej jako wynik klasyfikacji.
W ostantim kroku dataframe jest rozdzielany na dane treningowe i dane testowe.

In [50]:
# normalizacja danych

data = df[df.columns[0:19]]
target = df['Class']

col_names = ['Quality', 'Pre-screening', 'MA result1', 'MA result2', 'MA result3', 'MA result4', 'MA result5', 'MA result6', 'Exudates result1', 'Exudates result2', 'Exudates result3', 'Exudates result4', 'Exudates result5', 'Exudates result6', 'Exudates result7',
             'Exudates result', 'Macula - Optic disc dist', 'Optic disc diameter', 'AM/FM classification']
sc = StandardScaler()
data = sc.fit_transform(data)
data = pd.DataFrame(data, columns=col_names)
data

,Quality,Pre-screening,MA result1,MA result2,MA result3,MA result4,MA result5,MA result6,Exudates result1,Exudates result2,Exudates result3,Exudates result4,Exudates result5,Exudates result6,Exudates result7,Exudates result,Macula - Optic disc dist,Optic disc diameter,AM/FM classification
0,0.059054,0.298213,-0.641486,-0.618782,-0.576463,-0.630029,-0.551116,-0.473745,-0.242917,-0.246003,-0.296966,-0.271509,-0.218324,-0.194409,-0.205124,-0.186169,-1.294763,-0.468656,1.405048
1,0.059054,0.298213,-0.563391,-0.535778,-0.576463,-0.677410,-0.653676,-0.539992,-0.109250,0.032972,-0.465224,-0.408593,-0.224256,-0.197212,-0.205175,-0.186281,-0.082168,2.006054,-0.711719
2,0.059054,0.298213,0.920417,0.958299,1.046665,1.028299,0.936006,0.784951,-0.141383,0.227196,0.344463,0.769037,0.335538,0.152330,-0.110043,-0.164808,0.274283,1.121516,-0.711719
3,0.059054,0.298213,0.647084,0.667784,0.783456,0.838776,0.730886,0.652456,-0.404199,-0.214977,0.035830,0.316953,0.112573,0.056919,-0.195765,-0.199541,-1.423814,0.354501,-0.711719
4,0.059054,0.298213,0.217561,0.294265,0.388641,0.412349,0.525766,0.387468,-0.788069,-0.672306,-0.717335,-0.468311,-0.225828,-0.200905,-0.214968,-0.208100,-1.685874,0.844102,-0.711719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,0.059054,0.298213,-0.172915,-0.120756,-0.050043,0.033302,0.115525,0.188727,-0.992560,-1.025806,-0.750133,-0.467516,-0.225828,-0.200905,-0.214968,-0.208100,0.508422,0.466281,-0.711719
1147,0.059054,0.298213,0.412798,0.501776,0.607982,0.791395,0.833446,1.049939,-0.015387,0.198657,-0.055085,-0.218523,-0.225203,-0.200905,-0.214968,-0.208100,-0.231045,0.878556,-0.711719
1148,0.059054,-3.353309,0.412798,0.460274,0.564114,0.601872,0.730886,0.784951,-0.575348,-0.422401,-0.600326,-0.433156,-0.221309,-0.200905,-0.214968,-0.208100,1.334363,1.193713,-0.711719
1149,0.059054,0.298213,0.022323,-0.037752,-0.269384,-0.440506,-0.807516,-0.937474,-0.403199,-0.485477,-0.342804,-0.193527,0.001120,0.108426,0.386140,0.767878,-1.327962,-0.097078,1.405048


In [51]:
# !pip install feature_selector
# from feature_selector import FeatureSelector
# Features are in train and labels are in train_labels
# fs = FeatureSelector(data = data, labels = target)
# Pass in the appropriate parameters
# fs.identify_zero_importance(task = 'classification',
#                            eval_metric = 'auc',
#                            n_iterations = 10,
#                             early_stopping = True)
# list of zero importance features
# zero_importance_features = fs.ops['zero_importance']
# plot the feature importances
# fs.plot_feature_importances(threshold = 0.99, plot_n = 19)

In [52]:
# % chorych
target[target == 1].count() / target.count() * 100

53.08427454387489

In [53]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.20, random_state=10)

Pierwszym wykorzystanym algorytmem będzie naiwny klasyfikator bayesowski.

In [54]:
gnb = GaussianNB()
pred = gnb.fit(data_train, target_train).predict(data_test)
print("Naive-Bayes acc: ", accuracy_score(target_test, pred, normalize=True))

Naive-Bayes acc:  0.658008658008658


Wynik wychodzi słaby więc nie będzie to efektywny algorytm do tych danych.

Następnym sprawdzonym algorytmem będzie klasyfikacja liniowym wektorem nośnym

In [55]:
'''
BLOK WYKONUJE SIE OKOLO 3 MIN
'''
svc_model = LinearSVC(random_state=0, max_iter=10000000)

pred = svc_model.fit(data_train, target_train).predict(data_test)

print("LinearSVC acc: ", accuracy_score(target_test, pred, normalize=True))

LinearSVC acc:  0.7316017316017316


In [56]:
# data = data.drop(columns =['Macula - Optic disc dist', 'Optic disc diameter'])
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# data = sc.fit_transform(data)

Ostatnim sprawdzonym algorytmem będzie K najbliższych sąsiadów

In [58]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(data_train,  target_train)
pred = knn.predict(data_test)
print("KNN accuracy: ", accuracy_score(target_test, pred))

KNN accuracy:  0.6147186147186147


Precyzja jest znacznie lepsza od poprzednich algorytmów ale dalej wyniki nie są zadowalające.

In [45]:
# neural network


adam = Adam(lr=0.01)

classifier = Sequential()
classifier.add(Dense(5, activation='sigmoid',
                     kernel_initializer='random_normal', input_dim=19))
classifier.add(Dense(5, activation='sigmoid',
                     kernel_initializer='random_normal'))
classifier.add(Dense(5, activation='sigmoid',
                     kernel_initializer='random_normal'))
classifier.add(Dense(1, activation='sigmoid',
                     kernel_initializer='random_normal'))
classifier.compile(
    optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(data_train, target_train, epochs=1000)

prednn = classifier.predict(data_test)

Epoch 1/1000
920/920 [==============================] - 0s 108us/step - loss: 0.6929 - accuracy: 0.5087
Epoch 2/1000
920/920 [==============================] - 0s 19us/step - loss: 0.6933 - accuracy: 0.5217
Epoch 3/1000
920/920 [==============================] - 0s 19us/step - loss: 0.6923 - accuracy: 0.5217
Epoch 4/1000
920/920 [==============================] - 0s 18us/step - loss: 0.6906 - accuracy: 0.5217
Epoch 5/1000
920/920 [==============================] - 0s 16us/step - loss: 0.6764 - accuracy: 0.5446
Epoch 6/1000
920/920 [==============================] - 0s 16us/step - loss: 0.6358 - accuracy: 0.6533
Epoch 7/1000
920/920 [==============================] - 0s 16us/step - loss: 0.6046 - accuracy: 0.6859
Epoch 8/1000
920/920 [==============================] - 0s 16us/step - loss: 0.5921 - accuracy: 0.6913
Epoch 9/1000
920/920 [==============================] - 0s 16us/step - loss: 0.5873 - accuracy: 0.6989
Epoch 10/1000
920/920 [==============================] - 0s 16us/step - 

920/920 [==============================] - 0s 16us/step - loss: 0.4175 - accuracy: 0.8076
Epoch 158/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4137 - accuracy: 0.8033
Epoch 159/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4123 - accuracy: 0.8076
Epoch 160/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4170 - accuracy: 0.7978
Epoch 161/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4115 - accuracy: 0.8065
Epoch 162/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4164 - accuracy: 0.8109
Epoch 163/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4110 - accuracy: 0.8120
Epoch 164/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4107 - accuracy: 0.8098
Epoch 165/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4141 - accuracy: 0.8174
Epoch 166/1000
920/920 [==============================] - 0s 16us/step

920/920 [==============================] - 0s 16us/step - loss: 0.3921 - accuracy: 0.8207
Epoch 314/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4117 - accuracy: 0.8054
Epoch 315/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4062 - accuracy: 0.8022
Epoch 316/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4023 - accuracy: 0.8098
Epoch 317/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3956 - accuracy: 0.8033
Epoch 318/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4018 - accuracy: 0.8098
Epoch 319/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4097 - accuracy: 0.8141
Epoch 320/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4041 - accuracy: 0.8098
Epoch 321/1000
920/920 [==============================] - ETA: 0s - loss: 0.4647 - accuracy: 0.78 - 0s 16us/step - loss: 0.3938 - accuracy: 0.8109
Epoch 322/1000
920/920 [====

Epoch 390/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3923 - accuracy: 0.8207
Epoch 391/1000
920/920 [==============================] - ETA: 0s - loss: 0.4108 - accuracy: 0.81 - 0s 16us/step - loss: 0.3953 - accuracy: 0.8098
Epoch 392/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3910 - accuracy: 0.8196
Epoch 393/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3918 - accuracy: 0.8196
Epoch 394/1000
920/920 [==============================] - 0s 16us/step - loss: 0.4003 - accuracy: 0.8076
Epoch 395/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3975 - accuracy: 0.8130
Epoch 396/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3926 - accuracy: 0.8163
Epoch 397/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3942 - accuracy: 0.8163
Epoch 398/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3960 - accuracy: 0.8098
Epoch 399/100

920/920 [==============================] - 0s 16us/step - loss: 0.3852 - accuracy: 0.8163
Epoch 544/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3870 - accuracy: 0.8196
Epoch 545/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3846 - accuracy: 0.8217
Epoch 546/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3897 - accuracy: 0.8207
Epoch 547/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3855 - accuracy: 0.8217
Epoch 548/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3847 - accuracy: 0.8163
Epoch 549/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3897 - accuracy: 0.8109
Epoch 550/1000
920/920 [==============================] - 0s 17us/step - loss: 0.3882 - accuracy: 0.8174
Epoch 551/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3906 - accuracy: 0.8141
Epoch 552/1000
920/920 [==============================] - 0s 16us/step

920/920 [==============================] - 0s 16us/step - loss: 0.3751 - accuracy: 0.8185
Epoch 700/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3801 - accuracy: 0.8152
Epoch 701/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3778 - accuracy: 0.8207
Epoch 702/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3806 - accuracy: 0.8217
Epoch 703/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3769 - accuracy: 0.8196
Epoch 704/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3991 - accuracy: 0.8098
Epoch 705/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3903 - accuracy: 0.8152
Epoch 706/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3775 - accuracy: 0.8217
Epoch 707/1000
920/920 [==============================] - 0s 21us/step - loss: 0.3895 - accuracy: 0.8239
Epoch 708/1000
920/920 [==============================] - 0s 18us/step

920/920 [==============================] - 0s 16us/step - loss: 0.3734 - accuracy: 0.8272
Epoch 856/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3801 - accuracy: 0.8228
Epoch 857/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3761 - accuracy: 0.8261
Epoch 858/1000
920/920 [==============================] - 0s 17us/step - loss: 0.4045 - accuracy: 0.8043
Epoch 859/1000
920/920 [==============================] - 0s 18us/step - loss: 0.3825 - accuracy: 0.8261
Epoch 860/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3787 - accuracy: 0.8217
Epoch 861/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3763 - accuracy: 0.8185
Epoch 862/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3778 - accuracy: 0.8293
Epoch 863/1000
920/920 [==============================] - 0s 16us/step - loss: 0.3776 - accuracy: 0.8207
Epoch 864/1000
920/920 [==============================] - 0s 16us/step

In [46]:
prednn = (prednn > 0.5)
print("NN accuracy: ", accuracy_score(target_test, prednn))

NN accuracy:  0.7359307359307359


In [47]:
# DT

dt = tree.DecisionTreeClassifier()
dt.fit(data_train,  target_train)

pred = dt.predict(data_test)

print("DT accuracy: ", accuracy_score(target_test, pred))

DT accuracy:  0.5714285714285714


In [48]:
AdaBoost = AdaBoostClassifier(learning_rate=1)
AdaBoost.fit(data_train, target_train)
print(AdaBoost.score(data_test, target_test))
prediction_AdaBoost = AdaBoost.predict(data_test)

0.658008658008658


In [49]:
clf = MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=500, alpha=0.0001,
                    solver='sgd', verbose=10,  random_state=21, tol=0.000000001)
clf.fit(data_train, data_train)
pred = clf.predict(data_test)
print("MLP accuracy: ", accuracy_score(target_test, pred))

ValueError: Multioutput target data is not supported with label binarization